In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta

from geopy.distance import geodesic

In [2]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [3]:
wildfire_df = pd.read_sql("select * from wildfire_data",con=engine)
wildfire_df = wildfire_df[['fire_id','fire_name', 'discovery_date', 'containment_date', 'size', 'class', 'fire_year', 'lat', 'long']]
wildfire_df.head()

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year,lat,long
0,0,FOUNTAIN,2005-02-02,2005-02-02,0.10,A,2005,40.036944,-121.005833
1,1,PIGEON,2004-05-12,2004-05-12,0.25,A,2004,38.933056,-120.404444
2,2,SLACK,2004-05-31,2004-05-31,0.10,A,2004,38.984167,-120.735556
3,3,DEER,2004-06-28,2004-07-03,0.10,A,2004,38.559167,-119.913333
4,4,STEVENOT,2004-06-28,2004-07-03,0.10,A,2004,38.559167,-119.933056


In [4]:
wildfire_complete = wildfire_df.dropna()
#fill in missing values using average by each group
wildfire_complete["duration"] = (wildfire_complete["containment_date"] - wildfire_complete["discovery_date"]).dt.days
duration_class = wildfire_complete.groupby("class").agg({"duration":"mean"})
duration_class_dict = duration_class.to_dict()["duration"]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
duration_class_dict

{'A': 0.5803060142402666,
 'B': 0.8150366358712966,
 'C': 2.560782681099084,
 'D': 5.583904109589041,
 'E': 7.70935960591133,
 'F': 18.538709677419355,
 'G': 32.92018779342723}

In [6]:
wildfire_df[wildfire_df["containment_date"].isnull()].iloc[0:10,]

,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year,lat,long
821,821,PAINTED,2005-12-05,NaT,0.1,A,2005,39.409167,-120.541667
2035,2035,NEILSON,2006-08-23,NaT,7.0,B,2006,34.425000,-117.495833
2203,2203,WHEELER,2006-09-27,NaT,5.0,B,2006,38.513333,-119.988611
2235,2235,HARVEY,2006-06-26,NaT,1247.0,F,2006,40.058889,-122.908056
2294,2294,TITUS,2006-07-23,NaT,5792.0,G,2006,41.602778,-123.376944
2524,2524,BEAR,2006-08-22,NaT,0.1,A,2006,34.213056,-116.855556
2599,2599,SAND FLAT,2006-09-03,NaT,178.0,D,2006,38.403889,-119.779444
2600,2600,DEER,2006-07-21,NaT,0.1,A,2006,38.355833,-119.792222
2924,2924,BLUE,2006-09-29,NaT,0.5,B,2006,34.251667,-117.420833
3816,3816,WILLIS,2007-09-24,NaT,0.1,A,2007,41.685833,-123.665556


In [7]:
for index, row in wildfire_df.iterrows():
    if pd.isnull(row["containment_date"]):
        this_class = row["class"]
        wildfire_df.loc[index,"containment_date"] = row["discovery_date"] + pd.Timedelta(str(duration_class_dict[this_class]) + " days")

In [8]:
wildfire_df[wildfire_df["containment_date"].isnull()]

,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year,lat,long


In [9]:
wildfire_df.head()

,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year,lat,long
0,0,FOUNTAIN,2005-02-02,2005-02-02,0.10,A,2005,40.036944,-121.005833
1,1,PIGEON,2004-05-12,2004-05-12,0.25,A,2004,38.933056,-120.404444
2,2,SLACK,2004-05-31,2004-05-31,0.10,A,2004,38.984167,-120.735556
3,3,DEER,2004-06-28,2004-07-03,0.10,A,2004,38.559167,-119.913333
4,4,STEVENOT,2004-06-28,2004-07-03,0.10,A,2004,38.559167,-119.933056


In [10]:
# calculate the distance from san francisco based on lat & longitude


In [12]:
sf = (37.7749, -122.4194)
fire = (40.036944, -121.005833 )

In [13]:
geodesic(sf,fire).miles

173.6378146395041

In [15]:
wildfire_df.head()

,fire_id,fire_name,discovery_date,containment_date,size,class,fire_year,lat,long
0,0,FOUNTAIN,2005-02-02,2005-02-02,0.10,A,2005,40.036944,-121.005833
1,1,PIGEON,2004-05-12,2004-05-12,0.25,A,2004,38.933056,-120.404444
2,2,SLACK,2004-05-31,2004-05-31,0.10,A,2004,38.984167,-120.735556
3,3,DEER,2004-06-28,2004-07-03,0.10,A,2004,38.559167,-119.913333
4,4,STEVENOT,2004-06-28,2004-07-03,0.10,A,2004,38.559167,-119.933056


In [37]:
new_dict = {
    'name' : wildfire_df['fire_name'],
    'size' : wildfire_df['size'],
    'lat' : wildfire_df['lat'],
    'long' : wildfire_df['long'],
    'location': [wildfire_df['lat'], wildfire_df['long']]
}

In [54]:
new_df = pd.DataFrame(
    {'name': wildfire_df['fire_name'],
     'size': wildfire_df['size'],
     'distance' : geodesic((sf),(wildfire_df['lat'], wildfire_df['long'])).miles,
     'temp': wildfire_df['lat'] - wildfire_df['long']
    })

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [52]:
new_df.head()

,name,size,distance,temp
0,FOUNTAIN,0.10,0.0,161.042778
1,PIGEON,0.25,0.0,159.337500
2,SLACK,0.10,0.0,159.719722
3,DEER,0.10,0.0,158.472500
4,STEVENOT,0.10,0.0,158.492222


In [ ]:
import pandas as pd
data = [{'a': 1, 'b': 2},{'a': 5, 'b': 10, 'c': 20}]
df = pd.DataFrame(data)
print df

In [ ]:
wildfire_df.to_sql(name="cleaned_wildfire_data", con=engine, if_exists="replace", index=False)

In [ ]:
wildfire_df.head()